In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib
import math

# Data Collection and Dataframe Creation

In [2]:
pd.options.display.max_columns = None

In [3]:
# Get the league ID from the user
league_id = input("Enter the league ID: ")

# Create the URL using an f-string
url = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings/'

try:
    # Make the request and parse the response as JSON
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP request errors
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")
except requests.exceptions.JSONDecodeError:
    print("The response is not a valid JSON document.")

Enter the league ID:  414849


In [4]:
#Extract data for the DataFrame
new_entries = data.get('new_entries', {})
last_updated_data = data.get('last_updated_data', None)
league = data.get('league', {})
standings = data.get('standings', {}).get('results', [])

# Create DataFrames for each key if they are not empty
if new_entries:
    df_new_entries = pd.DataFrame(new_entries.get('results', []))
else:
    df_new_entries = pd.DataFrame()

if league:
    df_league = pd.DataFrame([league])  # Convert dictionary to a DataFrame with one row
else:
    df_league = pd.DataFrame()

if standings:
    df_standings = pd.DataFrame(standings)
else:
    df_standings = pd.DataFrame()

# Extract the columns of interest
columns_of_interest = ['entry', 'entry_name', 'rank', 'player_name']
df_filtered_standings = df_standings[columns_of_interest]

# Display the filtered standings DataFrame
print("Filtered Standings DataFrame:")
print(df_filtered_standings)

Filtered Standings DataFrame:
      entry            entry_name  rank      player_name
0    296696       The Special One     1      Theo Murray
1   2563745        Rice Rice Baby     2      Jack Murray
2   3190130               Macleod     3     Andy Macleod
3   2118759       The Hucklebucks     4     Simon Daniel
4    161957      poor last season     5     Ross Laverty
5   2078027       Rashford Fiesta     6  Ritchie Laverty
6   2082148   7th place last time     7     Louis Murray
7   1562151    MK Munchengladbach     8       Ken Murray
8   2077579    Grealish Lightning     9      Iain Murray
9   2683089     Super John McGinn    10  Stephen Macleod
10  2330200           Change Name    11      Euan Murray
11  1636610  We've got Egan-Riley    12   Daniel Macleod
12  2710147            Tubeventus    13     John Macleod
13  8011260         Cearns Cosmos    14    Gordon Murray
14  5642687     Harris St germain    15     John Macleod


In [5]:
# Function to get the history for a given team id
def fetch_team_history(team_id):
    url = f'https://fantasy.premierleague.com/api/entry/{team_id}/history/'
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP request errors
    return response.json()

# Initialize a list to store the results
all_histories = []

# Loop through each entry in the filtered standings DataFrame
for team_id in df_filtered_standings['entry']:
    try:
        history_data = fetch_team_history(team_id)
        # Flatten the JSON and add the team id for reference
        for history in history_data.get('current', []):
            history['entry'] = team_id
            all_histories.append(history)
    except requests.exceptions.RequestException as e:
        print(f"Request error for team id {team_id}: {e}")

# Create a new DataFrame from the collected history data
df_team_histories = pd.DataFrame(all_histories)

# Extract only the columns of interest
columns_of_interest = ['entry', 'event', 'points', 'total_points', 'value', 'points_on_bench']
df_team_histories_filtered = df_team_histories[columns_of_interest]

# Set 'entry' as the primary key
df_team_histories_filtered.set_index('entry', inplace=True)

# Display the new DataFrame
print("Filtered Team Histories DataFrame:")
print(df_team_histories_filtered)

Filtered Team Histories DataFrame:
         event  points  total_points  value  points_on_bench
entry                                                       
296696       1      73            73   1000                6
296696       2      40           113   1001               11
296696       3      45           158   1006                8
296696       4      87           245   1007                4
296696       5      64           309   1008                9
...        ...     ...           ...    ...              ...
5642687     34      62          1514    988                0
5642687     35      51          1565    987               10
5642687     36      78          1643    987                3
5642687     37      81          1724    986                8
5642687     38      39          1763    988                1

[570 rows x 5 columns]


In [6]:
# Extracting league name from df_league DataFrame
league_name = df_league.loc[0, 'name']

# Assign league name to variable 'League Name'
League_Name = league_name

# Displaying league name
print("League Name:", League_Name)

League Name: The worst league ever part 2


# Graph Creation

# PDF Creation

In [13]:
pip install reportlab

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 21.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [7]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
import pandas as pd

# Function to generate PDF
def generate_pdf(league_name, df_filtered_standings, df_team_histories_filtered):
    # Set up PDF document
    filename = f"{league_name}.pdf"
    doc = SimpleDocTemplate(filename, pagesize=letter)
    story = []

    # Title Page
    title_style = ParagraphStyle(name='Title', fontSize=18, alignment=1, spaceAfter=30)
    subtitle_style = ParagraphStyle(name='Subtitle', fontSize=14, alignment=1)

    title_text = "<br/><br/><br/><br/>Fantasy Premier League Analysis"
    subtitle_text = f"<br/><br/>{league_name}"

    title_paragraph = Paragraph(title_text, title_style)
    subtitle_paragraph = Paragraph(subtitle_text, subtitle_style)

    story.append(title_paragraph)
    story.append(subtitle_paragraph)
    story.append(canvas.Canvas(filename).showPage())

    # Table on Second Page
    data = []
    headers = ['Rank', 'Team Name', 'Total Points']
    
    # Prepare data for the table
    for index, row in df_filtered_standings.iterrows():
        team_id = row['entry']
        rank = row['rank']
        total_points = df_team_histories_filtered.loc[team_id, 'total_points']
        team_name = row['entry_name']
        
        data.append([rank, team_name, total_points])

    # Create table
    table = Table([headers] + data)
    table.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.gray),
                               ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                               ('GRID', (0, 0), (-1, -1), 1, colors.black)])
                  )
    story.append(table)

    # Build PDF document
    doc.build(story)
    print(f"PDF '{filename}' generated successfully.")

# Example usage:
if __name__ == "__main__":
    # Assuming df_filtered_standings and df_team_histories_filtered are already defined
    
    # Fetching League Name
    league_name = df_league.loc[0, 'name']

    # Generate PDF document
    generate_pdf(league_name, df_filtered_standings, df_team_histories_filtered)

PDF 'The worst league ever part 2.pdf' generated successfully.
